<a href="https://colab.research.google.com/github/msaantonova/IR-IE-project---Musicals-Recommendation-System/blob/main/Musicals_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IR/ IE Project** - Musical recommendation system

Workflow:

1. Finding the corpus

Possible corpus: https://docs.google.com/document/d/1TqIBg_3frwDtaE518Cq_AxAdtSsM93OWp8MTfJq0iPE/edit?tab=t.0

POr existing dataset with movie musicals reviews: https://www.kaggle.com/datasets/bwandowando/rotten-tomatoes-best-musicals-of-all-time/data
2. Indexing
3. Knowledge graphs (where do they go?)
4. Create query examples
5. Build an extraction pipeline for recognition of names etc.

**1. Working with the dataset**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import zipfile
import os
from google.colab import files

In [ ]:
file_path1 = '/content/movies.csv'
with open(file_path1, 'r', encoding='utf-8') as f:
    for i in range(5):
        print(f.readline())

"movieId","movieTitle","movieYear","movieURL","critic_score","audience_score"

"6da4d992-97ca-3acc-834c-37c1ad34872c","Singin' in the Rain",1952,"https://www.rottentomatoes.com/m/singin_in_the_rain","100%","95%"

"56b1624a-1ceb-3598-8ee2-62236554094a","Top Hat",1935,"https://www.rottentomatoes.com/m/top_hat","100%","90%"

"718dd15b-50ee-3090-84c7-13244b587b00","Meet Me in St. Louis",1944,"https://www.rottentomatoes.com/m/meet_me_in_st_louis","100%","87%"

"80f30474-3ac3-36c7-a43b-648d1a627432","The Wizard of Oz",1939,"https://www.rottentomatoes.com/m/the_wizard_of_oz_1939","98%","89%"



**`3. Knowledge graphs `**

In [ ]:
import networkx as nx

# Creating an emppty graph
G = nx.Graph()

# Adding nods for information
G.add_node("Les Misérables", type="musical")
G.add_node("The Phantom of the Opera", type="musical")
G.add_node("Hugh Jackman", type="actor")
G.add_node("Drama", type="genre")

# Adding edges for relationships
G.add_edge("Les Misérables", "Hugh Jackman", relationship="actor_in")
G.add_edge("Les Misérables", "Drama", relationship="has_genre")
G.add_edge("The Phantom of the Opera", "Hugh Jackman", relationship="actor_in")
G.add_edge("The Phantom of the Opera", "Drama", relationship="has_genre")

print(G.nodes)
print(G.edges)


['Les Misérables', 'The Phantom of the Opera', 'Hugh Jackman', 'Drama']
[('Les Misérables', 'Hugh Jackman'), ('Les Misérables', 'Drama'), ('The Phantom of the Opera', 'Hugh Jackman'), ('The Phantom of the Opera', 'Drama')]
